In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-l5l1is5t
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-l5l1is5t


In [2]:
!pip install -q transformers

In [4]:
from genericpath import exists
import sys
import os

my_path = '/content/bertsum1'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/MyDrive/ColabNotebooks/dacon-gas/AI/04/bertsum_realization', my_path)
sys.path.insert(0, my_path)

In [5]:
!pip install tensorboardX pyrouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import torch
import transformers

import datetime
import easydict
import itertools
import json
import matplotlib
import pathlib
import pprint
import re

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from operator import itemgetter
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from typing import List, Dict

## Version.
print("[VERSION]")
print(f"torch: {torch.__version__}")
print(f"transformers: {transformers.__version__}")

[VERSION]
torch: 1.11.0+cu113
transformers: 4.20.1


In [7]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
except:
    raise AssertionError()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!python -V

Python 3.7.13


In [9]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   39G   69G  37% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  812M  59% /sbin/docker-init
tmpfs           6.4G   28K  6.4G   1% /var/colab
/dev/sda1        81G   43G   39G  53% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive           108G   49G   60G  46% /content/drive


In [10]:
cd /content

/content


In [11]:
!nvidia-smi; free -h

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

              total        used        free      shared  buff/cache   available
Mem:            12G        734M        7.9G        1.2M        4.1G         11G
Swap:            0B          0B          0B


In [12]:
import os

model_flags = ['hidden_size', 'ff_size', 'heads', 'emb_size', 'enc_layers', 'enc_hidden_size', 'enc_ff_size',
               'dec_layers', 'dec_hidden_size', 'dec_ff_size', 'encoder', 'ff_actv', 'use_interval']

BERT_DATA_PATH='./drive/MyDrive/ColabNotebooks/dacon-gas/'
OUTPUT_PATH='./drive/MyDrive/ColabNotebooks/dacon-gas/bertsumabs/testest'
CHECKPOINT_PATH='./drive/MyDrive/ColabNotebooks/dacon-gas/pre/checkpoint/model_step_200000.pt'
LOGDIR='./logs/news.log'

args = easydict.EasyDict({
    "task": 'ext',
    "encoder": 'bert',
    "mode": 'train',
    "bert_data_path": './bert_data_kor/',
    "model_path": './models/',
    "result_path": './results/news',
    "temp_dir": './temp',

    "batch_size": 140,
    "test_batch_size": 200,

    "max_pos": 512,
    "use_interval":True,
    "large":False,
    "load_from_extractive": '',

    "sep_optim": False,
    "lr_bert": 2e-3,
    "lr_dec": 2e-3,
    "use_bert_emb": False,

    "share_emb":False,
    "finetune_bert": True,
    "dec_dropout": 0.2,
    "dec_layers": 6,
    "dec_hidden_size": 768 ,
    "dec_heads": 8 ,
    "dec_ff_size": 2048 ,
    "enc_hidden_size": 512 ,
    "enc_ff_size": 512 ,
    "enc_dropout": 0.2 ,
    "enc_layers": 6 ,

    # params for EXT
    "ext_dropout": 0.2 ,
    "ext_layers": 2 ,
    "ext_hidden_size": 768 ,
    "ext_heads": 8 ,
    "ext_ff_size": 2048 ,

    "label_smoothing": 0.1 ,
    "generator_shard_size": 32 ,
    "alpha":  0.6 ,
    "beam_size": 5 ,
    "min_length": 15 ,
    "max_length": 150 ,
    "max_tgt_len": 140 ,



    "param_init": 0 ,
    "param_init_glorot": True,
    "optim": 'adam' ,
    "lr": 1 ,
    "beta1":  0.9 ,
    "beta2": 0.999 ,
    "warmup_steps": 8000 ,
    "warmup_steps_bert": 8000 ,
    "warmup_steps_dec": 8000 ,
    "max_grad_norm": 0 ,

    "save_checkpoint_steps": 5 ,
    "accum_count": 1 ,
    "report_every": 1 ,
    "train_steps": 1000 ,
    "recall_eval":False,


    'visible_gpus': '-1' ,
    'gpu_ranks': '-1' ,
    'log_file': './logs/cnndm.log',
    'seed': 666 ,

    "test_all": False,
    "test_from": '',
    "test_start_from": -1 ,

    "train_from": '',
    "report_rouge": True,
    "block_trigram": True,
})

args.world_size = len(args.gpu_ranks)
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

In [13]:
args.task = 'abs'
args.mode = 'test'
args.batch_size = 3000
args.test_batch_size = 3000
args.bert_data_path = BERT_DATA_PATH
args.log_file = LOGDIR
args.sep_optim = True
args.use_interval = True
args.visible_gpus = '-1'
args.max_pos = 512
args.max_tgt_len = 300
args.alpha = 0.95
args.min_length = 50
args.report_rouge = False
args.result_path = OUTPUT_PATH
args.test_from = CHECKPOINT_PATH

In [14]:
!git clone https://github.com/hyunwoongko/kss

fatal: destination path 'kss' already exists and is not an empty directory.


In [15]:
cd /content/kss

/content/kss


In [16]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/kss
  Attempting uninstall: kss
    Found existing installation: kss 3.4.3
    Can't uninstall 'kss'. No files were found to uninstall.
  Running setup.py develop for kss


In [17]:
cd /content

/content


In [18]:
# !pip install -v python-mecab-ko==1.0.9

In [19]:
!pip install -q jupyter-dash==0.3.0 dash-bootstrap-components

In [20]:
!pip install -q dash==2.0.0

****

In [21]:
from train_abstractive import test_abs
from prepro.data_builder import BertData

In [22]:
ddd = BertData().preprocess('''
오늘(4일)도 전국 대부분 지역에서 무더운 날씨가 이어지겠습니다. 낮부터는 수도권과 중·남부 지역에 비 소식이 있습니다.

기상청은 이날 오전 전국 대부분 지역에 구름이 많고 남해안과 제주도는 대체로 흐리겠다고 예보했습니다.

낮 최고기온은 28~35도로 어제와 비슷하겠습니다.

체감온도 또한 33~35도 내외로 오르면서 매우 덥겠습니다.

이날 낮 12시부터 수도권과 남부지방 및 강원, 충청권에는 예상 강수량 5~40㎜의 소나기가 내릴 것으로 예상됩니다.

오전까지 중부 내륙에는 가시거리 1㎞ 미만의 안개가 낄 것으로 예보됐습니다.

제주도와 남해 먼바다에는 돌풍과 함께 천둥·번개가 치겠습니다. 바람이 시속 35~65㎞로 매우 강하게 불고 물결도 2~4m로 매우 높게 일겠습니다.

미세먼지 농도는 대기 확산이 원활하고 비가 내린 영향으로 전국이 대체로 '좋음' 또는 '보통' 수준을 보일 전망입니다.
''')

/content/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


[Korean Sentence Splitter]: Initializing Pynori...


In [23]:
pred = test_abs(args, ddd, device_id='-1', pt='', step=200000)

{'task': 'abs', 'encoder': 'bert', 'mode': 'test', 'bert_data_path': './drive/MyDrive/ColabNotebooks/dacon-gas/', 'model_path': './models/', 'result_path': './drive/MyDrive/ColabNotebooks/dacon-gas/bertsumabs/testest', 'temp_dir': './temp', 'batch_size': 3000, 'test_batch_size': 3000, 'max_pos': 512, 'use_interval': True, 'large': False, 'load_from_extractive': '', 'sep_optim': True, 'lr_bert': 0.002, 'lr_dec': 0.002, 'use_bert_emb': False, 'share_emb': False, 'finetune_bert': True, 'dec_dropout': 0.2, 'dec_layers': 6, 'dec_hidden_size': 768, 'dec_heads': 8, 'dec_ff_size': 2048, 'enc_hidden_size': 512, 'enc_ff_size': 512, 'enc_dropout': 0.2, 'enc_layers': 6, 'ext_dropout': 0.2, 'ext_layers': 2, 'ext_hidden_size': 768, 'ext_heads': 8, 'ext_ff_size': 2048, 'label_smoothing': 0.1, 'generator_shard_size': 32, 'alpha': 0.95, 'beam_size': 5, 'min_length': 50, 'max_length': 150, 'max_tgt_len': 300, 'param_init': 0, 'param_init_glorot': True, 'optim': 'adam', 'lr': 1, 'beta1': 0.9, 'beta2': 0.

In [24]:
pred

['기상청은 4일 오전 전국 대부분 지역에 구름이 많고 남해안과 제주도는 대체로 흐리겠다고 예보한 가운데 , 이날 낮 12시부터 수도권과 남부지방 및 강원권에는 예상 강수량 5 ~ 40mm의 소나기가 내릴 것으로 예상 강수량 5 ~ 4km 미만의 안개가 내릴 것으로 예상된다 .']

In [25]:
import time

import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import torch
def textbox(text, box="other"):
    style = {
        "max-width": "55%",
        "width": "max-content",
        "padding": "10px 15px",
        "border-radius": "25px"
    }

    if box == "self":
        style["margin-left"] = "auto"
        style["margin-right"] = 0

        color = "primary"
        inverse = True

    elif box == "other":
        style["margin-left"] = 0
        style["margin-right"] = "auto"

        color = "light"
        inverse = False

    else:
        raise ValueError("Incorrect option for `box`.")

    return dbc.Card(text, style=style, body=True, color=color, inverse=inverse)
conversation = html.Div(
    style={
        "width": "80%",
        "max-width": "800px",
        "height": "70vh",
        "margin": "auto",
        "overflow-y": "auto",
    },
    id="display-conversation",
)

controls = dbc.InputGroup(
    style={"width": "80%", "max-width": "800px", "margin": "auto"},
    children=[
        dbc.Input(id="user-input", placeholder="Write to the chatbot...", type="text"),
        dbc.Button("Submit", id="submit"),
    ],
)


# Define app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server


# Define Layout
app.layout = dbc.Container(
    fluid=True,
    style={'background-image': 'url(https://cdn.pixabay.com/photo/2020/05/19/13/54/man-5190962_960_720.jpg)'},
    children=[
        html.H1("BertAbsSum"),
        html.Hr(),
        dcc.Store(id="store-conversation", data=""),
        conversation,
        controls
    ],
)
@app.callback(
    Output("display-conversation", "children"), [Input("store-conversation", "data")]
)
def update_display(chat_history):
    return [
        textbox(x, box="self") if i % 2 == 0 else textbox(x, box="other")
        for i, x in enumerate(chat_history.split('<token>'))
    ]


@app.callback(
    [Output("store-conversation", "data"), Output("user-input", "value")],
    [Input("submit", "n_clicks"), Input("user-input", "n_submit")],
    [State("user-input", "value"), State("store-conversation", "data")],
)
def run_chatbot(n_clicks, n_submit, user_input, chat_history):
    if n_clicks == 0:
        return "", ""

    if user_input is None or user_input == "":
        return chat_history, ""

    bot_input_ids = BertData().preprocess(user_input)
        
    final_text = test_abs(args, bot_input_ids, -1, '', 200000)
    chat_history = user_input + '<token>' +final_text[0]
    return chat_history, ""

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """


In [26]:
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>